# 📌 **Credit Card Fraud Detection - Model Training & Evaluation**  

Fraud detection is a **complex classification task** due to the extreme class imbalance.  
This notebook focuses on **training and evaluating a deep learning model** to detect fraudulent transactions while minimizing false positives.  

---

## 🔹 **Workflow Overview**  

### 1️. Loading Preprocessed Data  
- Load datasets (`X_train`, `X_val`, `X_test`, `y_train`, `y_val`, `y_test`) from `.npy` files.  
- Verify the dataset structure before training.

### 2️. Defining Focal Loss for Handling Class Imbalance  
- Fraud detection datasets are highly **imbalanced**, making standard **binary cross-entropy loss ineffective**.  
- **Focal Loss** dynamically adjusts the importance of misclassified fraud cases.  

### 3️. Building the Neural Network Model  
- Construct a deep learning model using:
  - **Fully connected layers**
  - **Batch normalization**
  - **Dropout** for regularization.
- Implement **focal loss** to improve fraud detection.

### 4️. Compiling & Summarizing the Model  
- Define:
  - Optimizer: `Adam`
  - Loss function: `Focal Loss` or `Binary Crossentropy`
  - Metrics: **Precision**, **Recall**, **AUC**  
- Print the **model summary**.

### 5️. Training the Model  
- Train the model using **early stopping** to prevent overfitting.  
- Monitor `val_loss`, **precision**, and **recall**.

### 6️. Saving the Trained Model  
- Save the trained model to the **models/** directory for future use.  



## **1. Load Required Libraries**  

we need to import essential libraries for data processing, visualization, and model evaluation.

- `numpy` and `pandas` for efficient data manipulation.
- `seaborn` and `matplotlib` for exploratory data analysis (EDA).
- `scikit-learn` for data preprocessing, model evaluation, and performance metrics.
- `tensorflow` and `keras` for deep learning implementation.



In [ ]:
# 📌 Step 1: Load Required Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping


## **2. Load Preprocessed Data**  

Before training, we **load the preprocessed dataset** saved in the previous notebook.  
The dataset includes:  
- **Scaled features** to ensure numerical consistency.  
- **Balanced class labels** to mitigate the impact of class imbalance and improve fraud detection performance.  


In [8]:


# ✅ Define data paths
data_path = "/Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/data/processed/"

# ✅ Load preprocessed data
X_train = np.load(data_path + "X_train.npy")
X_val = np.load(data_path + "X_val.npy")
X_test = np.load(data_path + "X_test.npy")
y_train = np.load(data_path + "y_train.npy")
y_val = np.load(data_path + "y_val.npy")
y_test = np.load(data_path + "y_test.npy")

# ✅ Verify shapes
print("X_train Shape:", X_train.shape)
print("X_val Shape:", X_val.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_val Shape:", y_val.shape)
print("y_test Shape:", y_test.shape)


X_train Shape: (159491, 30)
X_val Shape: (39873, 30)
X_test Shape: (85443, 30)
y_train Shape: (159491,)
y_val Shape: (39873,)
y_test Shape: (85443,)


## **3. Implementing Focal Loss**  

Since **fraud cases are rare**, using standard **Binary Cross-Entropy loss** may cause the model to favor the majority class (non-fraudulent transactions).  

To address this, we implement **Focal Loss**, which dynamically adjusts the loss function to focus more on **misclassified fraud cases**, improving fraud detection performance.  


In [10]:
import tensorflow.keras.backend as K

def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal loss for binary classification.
    gamma > 1 penalizes easy examples
    alpha balances weighting of classes (0 < alpha < 1)
    """
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        epsilon = 1e-7
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        cross_entropy = -y_true * K.log(y_pred) - (1.0 - y_true) * K.log(1.0 - y_pred)
        weight = alpha * y_true * tf.pow((1.0 - y_pred), gamma) + \
                 (1.0 - alpha) * (1.0 - y_true) * tf.pow(y_pred, gamma)
        
        loss = weight * cross_entropy
        return tf.reduce_mean(loss)
    
    return focal_loss_fixed


## **4. Building and Compiling the Deep Learning Model**  

We build a **fully connected deep neural network** using the **Keras Sequential API**.  
The architecture consists of:  

- **Input Layer:** Accepts the preprocessed transaction data (`X_train`).  
- **Hidden Layers:**  
  - **Dense Layers** with **ReLU activation** to extract transaction patterns.  
  - **Batch Normalization** to stabilize learning and improve convergence.  
  - **Dropout Layers** (30% dropout) to prevent overfitting.  
- **Output Layer:**  
  - A **single neuron** with a **sigmoid activation function** for binary classification (Fraud/Not Fraud).  

This architecture ensures robust fraud detection while minimizing false positives.  

## 4.1. Compile the Model 

The model is compiled with one of the following loss functions:  
- **Focal Loss:** Adjusts the loss dynamically to focus more on misclassified fraud cases.  
- **Binary Crossentropy:** A standard loss function for binary classification.  

Additionally, we track the following performance metrics:  
- **Precision:** Measures how many predicted fraud cases are actually fraud.  
- **Recall:** Measures how many actual fraud cases are correctly identified.  
- **AUC (Area Under the Curve):** Evaluates the model's ability to distinguish between fraud and non-fraud transactions.  

These metrics provide a **balanced evaluation** of the model’s fraud detection capability.  


In [11]:
# ✅ Define Model
model = keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),  # Correct way to define input layer
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# ✅ Choose Loss Function
use_focal_loss = True

if use_focal_loss:
    loss_fn = focal_loss(gamma=2.0, alpha=0.25)
else:
    loss_fn = 'binary_crossentropy'

# ✅ Compile Model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=loss_fn,
    metrics=[
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')  # Adding AUC for better evaluation
    ]
)

# ✅ Print Model Summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 256)            │         7,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,945 (199.00 KB)

 Trainable params: 50,049 (195.50 KB)

 Non-trainable params: 896 (3.50 KB)

## **6. Train the Model**  

To optimize performance and prevent overfitting, we use **Early Stopping**, which:  
- Monitors the **validation loss (`val_loss`)** during training.  
- Stops training if the validation loss does not improve for a set number of epochs (**patience=5**).  
- Restores the best model weights to ensure optimal performance.  

The model is trained for **up to 50 epochs** with a **batch size of 512**, balancing efficiency and learning stability.  


In [12]:
# ✅ Define Early Stopping (to prevent overfitting)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

# ✅ Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,  # Start with 50, can adjust based on performance
    batch_size=512,  # Large batch sizes work well for fraud detection
    callbacks=[early_stopping]
)

# ✅ Save the Model
model.save("/Users/adityaiyer/Desktop/Credit-Card-Fraud-Detection/models/fraud_model.h5")

print("✅ Model training completed and saved successfully!")


Epoch 1/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - auc: 0.5966 - loss: 0.2713 - precision: 0.0022 - recall: 0.6029 - val_auc: 0.8322 - val_loss: 0.1245 - val_precision: 0.0033 - val_recall: 0.8116
Epoch 2/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - auc: 0.6301 - loss: 0.1384 - precision: 0.0028 - recall: 0.6033 - val_auc: 0.8302 - val_loss: 0.0784 - val_precision: 0.0405 - val_recall: 0.7971
Epoch 3/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - auc: 0.6277 - loss: 0.0876 - precision: 0.0056 - recall: 0.5437 - val_auc: 0.8330 - val_loss: 0.0527 - val_precision: 0.6707 - val_recall: 0.7971
Epoch 4/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - auc: 0.5058 - loss: 0.0584 - precision: 0.0093 - recall: 0.3680 - val_auc: 0.8403 - val_loss: 0.0338 - val_precision: 0.8594 - val_recall: 0.7971
Epoch 5/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - auc: 0.4915 - loss: 0.0394 - precision: 0.0317 - recall: 0.4035 - val_auc: 0.8394 - val_loss: 0.0226 - val_precision: 0.8833 - val_recall: 0.7681


✅ Model training completed and saved successfully!
